# Linear Transformation 
We will start with the alignement of the same models for different seeds. 
- First we do not restrict the matrix.
- Second we restrict the matrix to be a rotation matrix.
- Thrid use affine transformation 
- Last but not least we are using different norms and regularization techniques to improve the results



Steps: 
- Load the same model but with different seed
- Sample different images and get latent representation 
- Create Datamatrix X and X'
- Solve the simple optimization problem


In [1]:
# Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from sklearn.decomposition import PCA

from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt

/Users/federicoferoggio/Documents/vs_code/latent-communication/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration

config = {
    'path1': "/Users/federicoferoggio/Documents/vs_code/latent-communication/Pretrained_models/LightningAutoencoder_1.ckpt",
    'modelname1': 'AE1',
    'path2': "/Users/federicoferoggio/Documents/vs_code/latent-communication/Pretrained_models/LightningAutoencoder_1.ckpt",
    'modelname2': 'AE2',

}

# Optimization Problem in the Linear Case 
Let $x^i,y^i \in \mathbb{R^n}$ for $i = 1,...,m$ and $A \in \mathbb{R}^{n \times n}$ we are looking for the optimal A, which solves the following optimization problem 
$$ min_A \sum_{i = 1}^n ||Ax^i - y^i||^2 $$
where we are using the euclidian norm when not otherwise stated.



## Load Model and Transformed Data for VAE

In [3]:
# get working directory
import os
# Change directory
os.chdir('/Users/federicoferoggio/Documents/vs_code/latent-communication/')

# Import Data 
from helper.DataLoaderMNIST import DataLoader_MNIST

# Transdormations
transformations = [transforms.ToTensor(), 
                                # Normalize between -1 and 1
                                transforms.Normalize((0.5,), (0.5,)),
                                # Flatten the Image to a vector
                                transforms.Lambda(lambda x: x.view(-1) )
                                ]
# Load the data
data_loader = DataLoader_MNIST(128, transformations)

from AE.model_def import LightningAutoencoder

# Initialize the models
model1 = LightningAutoencoder()

# Load pretrained weights for model1
model1 = LightningAutoencoder.load_from_checkpoint(checkpoint_path=config['path1'])
# Initialize the model 2
model2 = LightningAutoencoder()
# Load pretrained weights for model2
model2 = LightningAutoencoder.load_from_checkpoint(checkpoint_path=config['path2'])

## Sampling 
We sample images from the train set and encode those for each model 

In [4]:
# Sample Size 
m = 100

# Sample images from train set 
images, _ = next(iter(data_loader.train_loader))


images = images.view(images.size(0), 1, 28, 28)
#
z1 = model1.getLatenSpace(images)
z2 = model2.getLatenSpace(images)

print(z1.shape)
print(z2.shape)
# Detach from GPU
z1 = z1.detach().cpu().numpy()
z2 = z2.detach().cpu().numpy()  

torch.Size([128, 500])
torch.Size([128, 500])


# Calulate optimal matrix 

In [16]:
# Regularization parameter
lamda = 0.0001

# Example data (replace with actual data)
z1 = np.random.rand(10, 5)
z2 = np.random.rand(10, 5)

# Get the dimensions of the latent space
m, d = z1.shape

# Initialize Gekko model
model = GEKKO(remote=False)

# Define the matrix variables in Gekko with initial guesses
A = model.Array(model.Var, (d, d), value=1.0)

# Objective function: Minimize sum of squared differences with regularization
objective = 0
for i in range(m):
    z1_transformed = [model.sum([A[j][k] * z1[i, k] for k in range(d)]) for j in range(d)]
    for j in range(d):
        objective += (z1_transformed[j] - z2[i, j])**2

# Regularization term (Frobenius norm)
reg_term = lamda * model.sqrt(model.sum([A[i][j]**2 for i in range(d) for j in range(d)]))

# Combine the objective and regularization term
model.Obj(objective + reg_term)

# Set the mode to steady-state optimization (mode 3)
model.options.SOLVER = 1  # IPOPT solver
model.solver_options = [
    'max_iter 1000',     # Maximum number of iterations
    'tol 1e-6',          # Tolerance
    'acceptable_tol 1e-5',  # Acceptable tolerance
    'acceptable_iter 10'  # Acceptable number of iterations
]

# Solve the optimization problem
model.solve(disp=True)

# Check if the solution is found and extract the optimized matrix A
if model.options.APPSTATUS == 1 or model.options.APPSTATUS == 2:
    A_opt = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            A_opt[i, j] = A[i][j].value[0]

    # Compute distance
    distance = np.zeros(m)
    for i in range(m):
        z1_transformed = np.dot(A_opt, z1[i, :])
        distance[i] = np.linalg.norm(z1_transformed - z2[i, :])

    # Plot the distance
    plt.hist(distance)
    plt.title('Histogram of distances')
    plt.xlabel('Distance')
    plt.ylabel('Frequency')
    plt.show()

    # Overall Loss 
    overall_loss = np.mean(distance)
    print(f'Overall Loss: {overall_loss}')
    print(f'Optimized Matrix A:\n{A_opt}')
else:
    print("Solution not found.")

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           51
   Constants    :            0
   Variables    :          351
   Intermediates:            0
   Connections  :          326
   Equations    :          276
   Residuals    :          276
 
 Number of state variables:            351
 Number of total equations: -          326
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :             25
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0          NaN  9.80733E-01
    1  6.63712E+00  7.43020E-01
    2 

Exception: @error: Solution Not Found


In [6]:

# Latent Space
latent_spaces1 = []
latent_spaces2 = []
all_labels = []

# Iterate through all batches in train_loader
for images, labels in data_loader.train_loader:
    images = images.view(images.size(0), 1, 28, 28)
    latent_space1 = model1.getLatenSpace(images)
    latent_space2 = model2.getLatenSpace(images)
    latent_spaces1.append(latent_space1.cpu().detach().numpy())
    latent_spaces2.append(latent_space2.cpu().detach().numpy())
    all_labels.append(labels.numpy())

# Concatenate latent space representations from all batches
latent_space1 = np.concatenate(latent_spaces1, axis=0)
latent_space2 = np.concatenate(latent_spaces2, axis=0)
all_labels = np.concatenate(all_labels, axis=0)


print(latent_space.shape)
print(all_labels.shape)



# Plot latent space via PCA
pca = PCA(n_components=2)


latent_space_pca = pca.fit_transform(latent_space)

# Plot the latent space
plot = plt.scatter(latent_space_pca[:, 0], latent_space_pca[:, 1], c=all_labels, cmap='tab10', label=all_labels)
plt.show(plot)

# Transform the latent space
latent_space_transformed = np.dot(latent_space, A.T)
# Plot latent space via PCA
pca = PCA(n_components=2)
latent_space_pca = pca.fit_transform(latent_space_transformed)

# Plot the latent space
plot = plt.scatter(latent_space_pca[:, 0], latent_space_pca[:, 1], c=all_labels, cmap='tab10', label=all_labels)
plt.show(plot)


# Plot the second latent space
latent_space_pca2 = pca.fit_transform(latent_space2)

pca_second = PCA(n_components=2)
plot_second = plt.scatter(latent_space_pca2[:, 0], latent_space_pca2[:, 1], c=all_labels, cmap='tab10', label=all_labels)
plt.show(plot_second)

NameError: name 'latent_space' is not defined